In [1]:
import numpy as np
import pandas as pd
import glob
import cv2
from ActiveContourDisc import activecontour
from ROI_from_excel import getROI
from ROI_from_excel import ShowResizedIm
from gutDWT import dwtopticdiscfinder
from cupB_eval import find_cup
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile

In [2]:
def getCDR(img,ROI_size):
    canvas = img.copy()
    ODC_flag = 0
    OD_flag = 0
    OC_flag = 0
    #1 img -> preprocess
    #2 proprocess -> coordinate
    xODC,yODC = dwtopticdiscfinder(img)
    xODC = int(xODC)
    yODC = int(yODC)
    if type(xODC) is not type(None) and type(yODC) is not type(None):
        ODC_flag = 1
    #3 coordinate -> ROI
    if ODC_flag:
        ROI_image,x_ROI1,y_ROI1,x_ROI2,y_ROI2 = getROI(img,xODC,yODC,ROI_size)
    #4 disc
    if ODC_flag:
        canvas,__,top_OD,__,bot_OD,snake = activecontour(canvas,ROI_image,xODC,
                                                   yODC,ROI_size,radius=250,B=90,WL=0.123,WE=6.5)
    if type(top_OD) is not type(None) and type(bot_OD) is not type(None) and type(snake) is not type(None):
        OD_flag = 1
        OD_size = round(bot_OD-top_OD,0)
    #5 cup
    if ODC_flag and OD_flag:
        ROI_imageL,x_ROI1,y_ROI1,x_ROI2,y_ROI2 = getROI(canvas,xODC,yODC,ROI_size)
        center, radious, area, error, image_result = find_cup(ROI_image,ROI_imageL)
    if type(center) is not type(None) and type(radious) is not type(None):
        OC_flag = 1
    if OC_flag:
        cv2.circle(canvas,(int(center[0]+x_ROI1),int(center[1]+y_ROI1)), radious,(0,255,0),2)
        OC_size = round(radious*2,0)
    #6 CDR
    if ODC_flag:
        cv2.line(canvas, (xODC+20, yODC), (xODC-20, yODC), (0, 0, 255), 3) 
        cv2.line(canvas, (xODC, yODC+20), (xODC, yODC-20), (0, 0, 255), 3) 
    if OD_flag and OC_flag:
        CDR = round(OC_size/OD_size,2)
        return canvas,OD_size,OC_size,CDR
    else:
        return canvas,None,None,None

In [3]:
def main():
    #=============USER INPUT=============
    
    crop_size = 500
    
    #=============USER END===============
    path = 'RAW_FUNDUS_INPUT/*.jpg'
    image_list = []
    for filename in glob.glob(path):
        image_list.append(filename)
    name_list = []
    OD_size_list = []
    OC_size_list = []
    CDR_list = []
    for i in image_list:
        img = cv2.imread(i) #import image
        output,OD_size,OC_size,CDR = getCDR(img,crop_size)
        name_list.append(i)
        if type(OD_size) is not type(None) and type(OC_size) is not type(None) and type(CDR) is not type(None):
            OD_size_list.append(OD_size)
            OC_size_list.append(OC_size)
            CDR_list.append(CDR)
        else:
            OD_size_list.append("None")
            OC_size_list.append("None")
            CDR_list.append("None")
        #ShowResizedIm(output,"image_result",2)
        #cv2.waitKey(0)
        #cv2.destroyAllWindows()
        print(i)
    df = pd.DataFrame({'file name':name_list,
                           'OD_size':OD_size_list,
                           'OC_size':OC_size_list,
                           'CDR':CDR_list
                          })
    writer = ExcelWriter('CDR.xlsx')
    df.to_excel(writer,'Sheet1',index=False)
    writer.save()
if __name__ == '__main__':
    main()

RAW_FUNDUS_INPUT\267_001.jpg
RAW_FUNDUS_INPUT\267_002.jpg
RAW_FUNDUS_INPUT\267_008.jpg
RAW_FUNDUS_INPUT\267_009.jpg
RAW_FUNDUS_INPUT\267_010.jpg
RAW_FUNDUS_INPUT\267_011.jpg
RAW_FUNDUS_INPUT\267_016.jpg
RAW_FUNDUS_INPUT\267_017.jpg
RAW_FUNDUS_INPUT\267_018.jpg
RAW_FUNDUS_INPUT\267_019.jpg
RAW_FUNDUS_INPUT\267_021.jpg
RAW_FUNDUS_INPUT\267_024.jpg
RAW_FUNDUS_INPUT\267_025.jpg
RAW_FUNDUS_INPUT\267_026.jpg
RAW_FUNDUS_INPUT\267_027.jpg
RAW_FUNDUS_INPUT\267_028.jpg
RAW_FUNDUS_INPUT\267_031.jpg
RAW_FUNDUS_INPUT\267_032.jpg
RAW_FUNDUS_INPUT\267_033.jpg
RAW_FUNDUS_INPUT\267_036.jpg
RAW_FUNDUS_INPUT\267_037.jpg
RAW_FUNDUS_INPUT\267_038.jpg
RAW_FUNDUS_INPUT\267_041.jpg
RAW_FUNDUS_INPUT\267_042.jpg
RAW_FUNDUS_INPUT\267_045.jpg
RAW_FUNDUS_INPUT\267_046.jpg
RAW_FUNDUS_INPUT\267_049.jpg
RAW_FUNDUS_INPUT\267_050.jpg
RAW_FUNDUS_INPUT\267_051.jpg
RAW_FUNDUS_INPUT\267_053.jpg
RAW_FUNDUS_INPUT\267_056.jpg
RAW_FUNDUS_INPUT\267_064.jpg
RAW_FUNDUS_INPUT\267_067.jpg
RAW_FUNDUS_INPUT\267_068.jpg
RAW_FUNDUS_INP

RAW_FUNDUS_INPUT\272_089.jpg
RAW_FUNDUS_INPUT\272_090.jpg
RAW_FUNDUS_INPUT\272_091.jpg
RAW_FUNDUS_INPUT\272_092.jpg
RAW_FUNDUS_INPUT\272_093.jpg
RAW_FUNDUS_INPUT\272_094.jpg
RAW_FUNDUS_INPUT\272_095.jpg
RAW_FUNDUS_INPUT\272_096.jpg
RAW_FUNDUS_INPUT\272_097.jpg
RAW_FUNDUS_INPUT\272_098.jpg
RAW_FUNDUS_INPUT\277_001.jpg
RAW_FUNDUS_INPUT\277_002.jpg
RAW_FUNDUS_INPUT\277_003.jpg
RAW_FUNDUS_INPUT\277_005.jpg
RAW_FUNDUS_INPUT\277_008.jpg
RAW_FUNDUS_INPUT\277_009.jpg
RAW_FUNDUS_INPUT\277_011.jpg
RAW_FUNDUS_INPUT\277_013.jpg
RAW_FUNDUS_INPUT\277_014.jpg
RAW_FUNDUS_INPUT\277_016.jpg
RAW_FUNDUS_INPUT\277_017.jpg
RAW_FUNDUS_INPUT\277_018.jpg
RAW_FUNDUS_INPUT\277_019.jpg
RAW_FUNDUS_INPUT\277_020.jpg
RAW_FUNDUS_INPUT\277_021.jpg
RAW_FUNDUS_INPUT\277_022.jpg
RAW_FUNDUS_INPUT\277_023.jpg
RAW_FUNDUS_INPUT\277_024.jpg
RAW_FUNDUS_INPUT\277_025.jpg
RAW_FUNDUS_INPUT\277_026.jpg
RAW_FUNDUS_INPUT\277_027.jpg
RAW_FUNDUS_INPUT\277_028.jpg
RAW_FUNDUS_INPUT\277_029.jpg
RAW_FUNDUS_INPUT\277_030.jpg
RAW_FUNDUS_INP

<Figure size 432x288 with 0 Axes>